# Review of Tensors

In this notebook, we will cover:

* Introduction to tensors
* Getting info from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try


In [1]:
import tensorflow as tf
print(tf.__version__)

2.18.0


Create tensors with `tf.constant()`

In [2]:
scalar = tf.constant(7)
vector = tf.constant([10, 10])
matrix = tf.constant([[10, 7],
                      [7, 10]])
scalar,vector,matrix

(<tf.Tensor: shape=(), dtype=int32, numpy=7>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[10,  7],
        [ 7, 10]], dtype=int32)>)

Check the number of dimensions of a tensor.

In [3]:
scalar.ndim,vector.ndim,matrix.ndim

(0, 1, 2)

In [4]:
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the datatype with 'dtype'
another_matrix, another_matrix.ndim

(<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
 array([[10.,  7.],
        [ 3.,  2.],
        [ 8.,  9.]], dtype=float16)>,
 2)

So, `another_matrix` is a 2-dimensional tensor with 3 elements.

In [5]:
tensor = tf.constant([
                        [
                            [10., 7., 2.],
                            [3., 2., 5.]
                        ],

                        [
                            [8., 9., 11.],
                            [7., 9., 15,]
                        ],

                        [
                            [3., 5., 1.],
                            [9.,6.,4.]
                        ]

                    ], dtype=tf.float16) # specify the datatype with 'dtype'
tensor, tensor.ndim

(<tf.Tensor: shape=(3, 2, 3), dtype=float16, numpy=
 array([[[10.,  7.,  2.],
         [ 3.,  2.,  5.]],
 
        [[ 8.,  9., 11.],
         [ 7.,  9., 15.]],
 
        [[ 3.,  5.,  1.],
         [ 9.,  6.,  4.]]], dtype=float16)>,
 3)

Changeable tensor with `tf.Variable()`

In [6]:
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

Change value in `changeable_tensor` using `.assign()`

In [7]:
changeable_tensor[0].assign([7])
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

👉 Constant tensors can only be changed by re-assignment

In [8]:
unchangeable_tensor = tf.constant([7,7])
unchangeable_tensor

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 7], dtype=int32)>

#### Creating Random Tensors

From normal distribution ($\mu = 0, \sigma = 1$)

In [9]:
random_1 = tf.random.Generator.from_seed(42)
random_1 = random_1.normal(shape=(3,2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

From uniform distribution

In [10]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.uniform(shape=(3,2))
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.7493447 , 0.73561966],
       [0.45230794, 0.49039817],
       [0.1889317 , 0.52027524]], dtype=float32)>

#### Shuffle the order of elements in a tensor

In [11]:
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [12]:
tf.random.set_seed(42)   # need to specify seed here
shuffled = tf.random.shuffle(not_shuffled, seed=42)  # need to specify seed here too.
shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

#### Other ways to create tensors

In [13]:
tf.ones(shape=(10,7))

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [14]:
tf.zeros(shape=(3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

Create tensors from Numpy arrays.

👉 Tensors are designed to run on GPU computing.

In [15]:
import numpy as np

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(some_vector) # non-capital for vector

numpy_A = np.arange(1, 25, dtype=np.int32) # create a Numpy array between 1 and 25
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [16]:
tensor_A1 = tf.constant(numpy_A)
tensor_A1

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [17]:
tensor_A2 = tf.constant(numpy_A, shape=(3,8))
tensor_A2

<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
array([[ 1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16],
       [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>

In [18]:
tensor_A3 = tf.constant(numpy_A, shape=(2, 3, 4))
tensor_A3


<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]], dtype=int32)>

In [19]:
tensor_A4 = tf.constant(numpy_A, shape=(2, 2, 3, 2))
tensor_A4

<tf.Tensor: shape=(2, 2, 3, 2), dtype=int32, numpy=
array([[[[ 1,  2],
         [ 3,  4],
         [ 5,  6]],

        [[ 7,  8],
         [ 9, 10],
         [11, 12]]],


       [[[13, 14],
         [15, 16],
         [17, 18]],

        [[19, 20],
         [21, 22],
         [23, 24]]]], dtype=int32)>

#### Getting stats from tensors (shape, rank, dimension, size)



In [23]:
rank_4_tensor = tf.random.Generator.from_seed(42)
rank_4_tensor = rank_4_tensor.uniform(shape=(2,3,4,5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0.7493447 , 0.73561966, 0.45230794, 0.49039817, 0.1889317 ],
         [0.52027524, 0.8736881 , 0.46921718, 0.63932586, 0.6467117 ],
         [0.96246755, 0.41009164, 0.86540747, 0.8862978 , 0.27795732],
         [0.8857763 , 0.2179842 , 0.29115117, 0.03953862, 0.8136791 ]],

        [[0.8139852 , 0.52180684, 0.12496924, 0.5488483 , 0.7755773 ],
         [0.6184403 , 0.24936223, 0.89341843, 0.28422844, 0.70332646],
         [0.2622137 , 0.4432162 , 0.466465  , 0.05981874, 0.40098202],
         [0.69292355, 0.1284684 , 0.22770369, 0.33691216, 0.5329138 ]],

        [[0.5914326 , 0.21738243, 0.5322075 , 0.05148339, 0.03951418],
         [0.41866875, 0.78939915, 0.04384279, 0.96955836, 0.49116182],
         [0.3706199 , 0.33535397, 0.02760839, 0.99757504, 0.52172756],
         [0.20307171, 0.7182547 , 0.84263575, 0.9542595 , 0.90112185]]],


       [[[0.62839293, 0.08619452, 0.5679928 , 0.28763676, 0.18828917],
         [0.665

In [25]:
rank_4_tensor[1]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0.62839293, 0.08619452, 0.5679928 , 0.28763676, 0.18828917],
        [0.6658715 , 0.6190208 , 0.04422736, 0.49626625, 0.6474861 ],
        [0.3557682 , 0.32033885, 0.33705008, 0.7391062 , 0.16512096],
        [0.41555858, 0.78638244, 0.45899415, 0.42686343, 0.5630441 ]],

       [[0.04115057, 0.7532015 , 0.18315244, 0.30807555, 0.30482936],
        [0.08611703, 0.7526083 , 0.42415357, 0.7877505 , 0.11611497],
        [0.35255086, 0.50718856, 0.16467738, 0.54108894, 0.5765736 ],
        [0.16844285, 0.8007604 , 0.34483027, 0.30735934, 0.9700769 ]],

       [[0.43336582, 0.51829636, 0.8555572 , 0.21962428, 0.76091194],
        [0.04762888, 0.5131633 , 0.97339594, 0.33341944, 0.03152311],
        [0.28874612, 0.58736026, 0.75669694, 0.7060174 , 0.5588795 ],
        [0.40088904, 0.8377521 , 0.7651175 , 0.5402924 , 0.52445555]]],
      dtype=float32)>

In [26]:
rank_4_tensor.shape,rank_4_tensor.ndim,tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [27]:
rank_4_tensor[:2,:3].shape,rank_4_tensor[:2,:3].ndim,tf.size(rank_4_tensor[:2,:3])

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

Get various attributes from `rank_4_tensor`:


In [28]:
print("Data type of every element", rank_4_tensor.dtype)
print("\nNumber of dimensions (rank)", rank_4_tensor.ndim)
print("\nShape of tensor", rank_4_tensor.shape)
print("\nElements along axis 0 of tensor:\n", rank_4_tensor[0])
print("\nElements along last axis of tensor:\n", rank_4_tensor[-1])
print("\nTotal number of elements in tensor:\n", tf.size(rank_4_tensor).numpy())

Data type of every element <dtype: 'float32'>

Number of dimensions (rank) 4

Shape of tensor (2, 3, 4, 5)

Elements along axis 0 of tensor:
 tf.Tensor(
[[[0.7493447  0.73561966 0.45230794 0.49039817 0.1889317 ]
  [0.52027524 0.8736881  0.46921718 0.63932586 0.6467117 ]
  [0.96246755 0.41009164 0.86540747 0.8862978  0.27795732]
  [0.8857763  0.2179842  0.29115117 0.03953862 0.8136791 ]]

 [[0.8139852  0.52180684 0.12496924 0.5488483  0.7755773 ]
  [0.6184403  0.24936223 0.89341843 0.28422844 0.70332646]
  [0.2622137  0.4432162  0.466465   0.05981874 0.40098202]
  [0.69292355 0.1284684  0.22770369 0.33691216 0.5329138 ]]

 [[0.5914326  0.21738243 0.5322075  0.05148339 0.03951418]
  [0.41866875 0.78939915 0.04384279 0.96955836 0.49116182]
  [0.3706199  0.33535397 0.02760839 0.99757504 0.52172756]
  [0.20307171 0.7182547  0.84263575 0.9542595  0.90112185]]], shape=(3, 4, 5), dtype=float32)

Elements along last axis of tensor:
 tf.Tensor(
[[[0.62839293 0.08619452 0.5679928  0.28763676 0.18

#### Indexing Tensors

In [29]:
# Get the first two elements of each dimension

rank_4_tensor[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0.7493447 , 0.73561966],
         [0.52027524, 0.8736881 ]],

        [[0.8139852 , 0.52180684],
         [0.6184403 , 0.24936223]]],


       [[[0.62839293, 0.08619452],
         [0.6658715 , 0.6190208 ]],

        [[0.04115057, 0.7532015 ],
         [0.08611703, 0.7526083 ]]]], dtype=float32)>

In [30]:
rank_4_tensor[:1,:2,:2,:2]

<tf.Tensor: shape=(1, 2, 2, 2), dtype=float32, numpy=
array([[[[0.7493447 , 0.73561966],
         [0.52027524, 0.8736881 ]],

        [[0.8139852 , 0.52180684],
         [0.6184403 , 0.24936223]]]], dtype=float32)>

In [31]:
# Get the first element from each dimension except the last one.
rank_4_tensor[:1,:1,:1,:]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=
array([[[[0.7493447 , 0.73561966, 0.45230794, 0.49039817, 0.1889317 ]]]],
      dtype=float32)>

In [32]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.random.Generator.from_seed(42)
rank_2_tensor = rank_2_tensor.uniform(shape=(2,3))
rank_2_tensor

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.7493447 , 0.73561966, 0.45230794],
       [0.49039817, 0.1889317 , 0.52027524]], dtype=float32)>

In [33]:
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 3]), 2)

In [34]:
# Get the last element of each row
rank_2_tensor[:,-1]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.45230794, 0.52027524], dtype=float32)>

In [35]:
# Add a new dimension to the rank_2_tensor
rank_2_tensor_new_dim = rank_2_tensor[...,tf.newaxis]
rank_2_tensor_new_dim

<tf.Tensor: shape=(2, 3, 1), dtype=float32, numpy=
array([[[0.7493447 ],
        [0.73561966],
        [0.45230794]],

       [[0.49039817],
        [0.1889317 ],
        [0.52027524]]], dtype=float32)>

In [36]:
# Another way of adding a new dimension
tf.expand_dims(rank_2_tensor, axis=-1)

<tf.Tensor: shape=(2, 3, 1), dtype=float32, numpy=
array([[[0.7493447 ],
        [0.73561966],
        [0.45230794]],

       [[0.49039817],
        [0.1889317 ],
        [0.52027524]]], dtype=float32)>

In [37]:
tf.expand_dims(rank_2_tensor, axis=0)

<tf.Tensor: shape=(1, 2, 3), dtype=float32, numpy=
array([[[0.7493447 , 0.73561966, 0.45230794],
        [0.49039817, 0.1889317 , 0.52027524]]], dtype=float32)>

In [38]:
tf.expand_dims(rank_2_tensor, axis=1)


<tf.Tensor: shape=(2, 1, 3), dtype=float32, numpy=
array([[[0.7493447 , 0.73561966, 0.45230794]],

       [[0.49039817, 0.1889317 , 0.52027524]]], dtype=float32)>

#### Tensor Operations

In [39]:
tensor = tf.constant([[10,7],[3,4]])
print(tensor+6,"\n\n", tensor-6,"\n\n", tensor*6,"\n\n", tensor/6)

tf.Tensor(
[[16 13]
 [ 9 10]], shape=(2, 2), dtype=int32) 

 tf.Tensor(
[[ 4  1]
 [-3 -2]], shape=(2, 2), dtype=int32) 

 tf.Tensor(
[[60 42]
 [18 24]], shape=(2, 2), dtype=int32) 

 tf.Tensor(
[[1.66666667 1.16666667]
 [0.5        0.66666667]], shape=(2, 2), dtype=float64)


We can use Tensorflow's built-in functions too.

In [40]:
print(tf.add(tensor, 6),"\n\n", tf.subtract(tensor, 6),"\n\n", tf.multiply(tensor, 6),"\n\n", tf.divide(tensor, 6))

tf.Tensor(
[[16 13]
 [ 9 10]], shape=(2, 2), dtype=int32) 

 tf.Tensor(
[[ 4  1]
 [-3 -2]], shape=(2, 2), dtype=int32) 

 tf.Tensor(
[[60 42]
 [18 24]], shape=(2, 2), dtype=int32) 

 tf.Tensor(
[[1.66666667 1.16666667]
 [0.5        0.66666667]], shape=(2, 2), dtype=float64)


#### Matrix Multiplication

In [41]:
print(tensor, "\n\n",tensor*tensor,"\n\n",tensor*tensor*tensor ) # element-wise multiplication

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32) 

 tf.Tensor(
[[100  49]
 [  9  16]], shape=(2, 2), dtype=int32) 

 tf.Tensor(
[[1000  343]
 [  27   64]], shape=(2, 2), dtype=int32)


In [42]:
tf.matmul(tensor, tensor)  # dot product multiplication

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [43]:
tensor @ tensor # dot product multiplication

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

Multiplying matrices with different shapes  $(n,m) \cdot (m,p) = (n,p)$

In [44]:
tensor2 = tf.constant([[1,2,3],[4,5,6]])
tensor2

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

In [45]:
tensor3 = tf.constant([[7,8],[9,10],[11,12]])
tensor3

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 7,  8],
       [ 9, 10],
       [11, 12]], dtype=int32)>

In [46]:
tensor3 @ tensor

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 94,  81],
       [120, 103],
       [146, 125]], dtype=int32)>

In [47]:
# Create (3, 2) tensor
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])

# Create another (3, 2) tensor
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [48]:
tf.matmul(X,Y)   # incompatible matrix shapes, we get an error.

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul] name: 

Reshaping matrices with `tf.reshape()`

In [49]:
Y_reshape = tf.reshape(Y, shape=(2,3))
Y_reshape

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [50]:
tf.matmul(X,Y_reshape)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [51]:
tf.tensordot(X,Y_reshape, axes=1)


<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

👉 **Note:** Reshaping is NOT the same as transposing!

In [52]:
X,Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [53]:
tf.matmul(X,tf.transpose(Y)), tf.matmul(X,tf.reshape(Y, shape=(2,3)))

(<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
 array([[ 23,  29,  35],
        [ 53,  67,  81],
        [ 83, 105, 127]], dtype=int32)>,
 <tf.Tensor: shape=(3, 3), dtype=int32, numpy=
 array([[ 27,  30,  33],
        [ 61,  68,  75],
        [ 95, 106, 117]], dtype=int32)>)

In [54]:
Y, tf.transpose(Y), tf.reshape(Y, shape=(2,3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[ 7,  9, 11],
        [ 8, 10, 12]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[ 7,  8,  9],
        [10, 11, 12]], dtype=int32)>)

#### Changing the data type of a tensor

In [55]:
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0.7493447 , 0.73561966, 0.45230794, 0.49039817, 0.1889317 ],
         [0.52027524, 0.8736881 , 0.46921718, 0.63932586, 0.6467117 ],
         [0.96246755, 0.41009164, 0.86540747, 0.8862978 , 0.27795732],
         [0.8857763 , 0.2179842 , 0.29115117, 0.03953862, 0.8136791 ]],

        [[0.8139852 , 0.52180684, 0.12496924, 0.5488483 , 0.7755773 ],
         [0.6184403 , 0.24936223, 0.89341843, 0.28422844, 0.70332646],
         [0.2622137 , 0.4432162 , 0.466465  , 0.05981874, 0.40098202],
         [0.69292355, 0.1284684 , 0.22770369, 0.33691216, 0.5329138 ]],

        [[0.5914326 , 0.21738243, 0.5322075 , 0.05148339, 0.03951418],
         [0.41866875, 0.78939915, 0.04384279, 0.96955836, 0.49116182],
         [0.3706199 , 0.33535397, 0.02760839, 0.99757504, 0.52172756],
         [0.20307171, 0.7182547 , 0.84263575, 0.9542595 , 0.90112185]]],


       [[[0.62839293, 0.08619452, 0.5679928 , 0.28763676, 0.18828917],
         [0.665

In [56]:
tf.cast(rank_4_tensor, dtype=tf.float16)

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float16, numpy=
array([[[[0.7495 , 0.736  , 0.4524 , 0.4905 , 0.189  ],
         [0.5205 , 0.8735 , 0.4692 , 0.639  , 0.6465 ],
         [0.9624 , 0.4102 , 0.865  , 0.886  , 0.278  ],
         [0.8857 , 0.218  , 0.2913 , 0.03955, 0.8135 ]],

        [[0.814  , 0.522  , 0.12494, 0.549  , 0.7754 ],
         [0.6187 , 0.2494 , 0.8936 , 0.2842 , 0.703  ],
         [0.2622 , 0.443  , 0.4666 , 0.0598 , 0.401  ],
         [0.693  , 0.1284 , 0.2277 , 0.337  , 0.5327 ]],

        [[0.5913 , 0.2174 , 0.532  , 0.05148, 0.03952],
         [0.4187 , 0.7896 , 0.04385, 0.9697 , 0.4912 ],
         [0.3706 , 0.3354 , 0.0276 , 0.9976 , 0.5215 ],
         [0.2031 , 0.7183 , 0.843  , 0.954  , 0.901  ]]],


       [[[0.6284 , 0.0862 , 0.568  , 0.2876 , 0.1882 ],
         [0.666  , 0.619  , 0.04422, 0.4963 , 0.6475 ],
         [0.3557 , 0.3203 , 0.3372 , 0.7393 , 0.1652 ],
         [0.4155 , 0.7866 , 0.459  , 0.4268 , 0.563  ]],

        [[0.04114, 0.7534 , 0.1831 , 0.3

In [57]:
tf.cast(rank_4_tensor, dtype=tf.int64)

<tf.Tensor: shape=(2, 3, 4, 5), dtype=int64, numpy=
array([[[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]]])>

In [58]:
tensor_A1, tf.cast(tensor_A1, dtype=tf.float16)

(<tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=float16, numpy=
 array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.],
       dtype=float16)>)

#### Aggregating Tensors

In [59]:
# absolute value

random_1, tf.abs(random_1)

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.7565803 , 0.06854702],
        [0.07595026, 1.2573844 ],
        [0.23193763, 1.8107855 ]], dtype=float32)>)

In [60]:
# Min, Max, Mean, Sum, Std_Dev

print(rank_2_tensor,"\n\n min:",
      tf.reduce_min(rank_2_tensor,keepdims=True),"\n\n max:",
      tf.reduce_max(rank_2_tensor,keepdims=True),"\n\n mean:",
      tf.reduce_mean(rank_2_tensor,keepdims=True),"\n\n sum:",
      tf.reduce_sum(rank_2_tensor,keepdims=True), "\n\n std_dev:",
      tf.math.reduce_std(tf.cast(rank_2_tensor, dtype=tf.float32)),"\n\n variance:",
      tf.math.reduce_variance(tf.cast(rank_2_tensor, dtype=tf.float32)))

tf.Tensor(
[[0.7493447  0.73561966 0.45230794]
 [0.49039817 0.1889317  0.52027524]], shape=(2, 3), dtype=float32) 

 min: tf.Tensor([[0.1889317]], shape=(1, 1), dtype=float32) 

 max: tf.Tensor([[0.7493447]], shape=(1, 1), dtype=float32) 

 mean: tf.Tensor([[0.5228129]], shape=(1, 1), dtype=float32) 

 sum: tf.Tensor([[3.1368775]], shape=(1, 1), dtype=float32) 

 std_dev: tf.Tensor(0.18890391, shape=(), dtype=float32) 

 variance: tf.Tensor(0.03568469, shape=(), dtype=float32)


#### Find the positional min and max in a tensor

In [61]:
rank_1_tensor = tf.random.Generator.from_seed(42)
rank_1_tensor = rank_1_tensor.uniform(shape=(50,))
rank_1_tensor, rank_1_tensor.ndim

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.7493447 , 0.73561966, 0.45230794, 0.49039817, 0.1889317 ,
        0.52027524, 0.8736881 , 0.46921718, 0.63932586, 0.6467117 ,
        0.96246755, 0.41009164, 0.86540747, 0.8862978 , 0.27795732,
        0.8857763 , 0.2179842 , 0.29115117, 0.03953862, 0.8136791 ,
        0.8139852 , 0.52180684, 0.12496924, 0.5488483 , 0.7755773 ,
        0.6184403 , 0.24936223, 0.89341843, 0.28422844, 0.70332646,
        0.2622137 , 0.4432162 , 0.466465  , 0.05981874, 0.40098202,
        0.69292355, 0.1284684 , 0.22770369, 0.33691216, 0.5329138 ,
        0.5914326 , 0.21738243, 0.5322075 , 0.05148339, 0.03951418,
        0.41866875, 0.78939915, 0.04384279, 0.96955836, 0.49116182],
       dtype=float32)>,
 1)

In [62]:
tf.argmin(rank_1_tensor),tf.argmax(rank_1_tensor)

(<tf.Tensor: shape=(), dtype=int64, numpy=44>,
 <tf.Tensor: shape=(), dtype=int64, numpy=48>)

In [63]:
print(rank_2_tensor,"\n\n argmin:", tf.argmin(rank_2_tensor),"\n\n argmax:",tf.argmax(rank_2_tensor))

tf.Tensor(
[[0.7493447  0.73561966 0.45230794]
 [0.49039817 0.1889317  0.52027524]], shape=(2, 3), dtype=float32) 

 argmin: tf.Tensor([1 1 0], shape=(3,), dtype=int64) 

 argmax: tf.Tensor([0 0 1], shape=(3,), dtype=int64)
